This script was made based on Tensorflow's tutorial: https://www.tensorflow.org/tutorials/load_data/images

In [ ]:
# imports

from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

tf.enable_eager_execution()  # enables imperative execution without building tensor graphs (debug mode)

AUTOTUNE = tf.data.experimental.AUTOTUNE  # an alias to write less
EPOCHS = 100
IMG_W = 96
IMG_H = 96
TRAIN_TEST_RATIO = 0.8

In [ ]:
data_root = pathlib.Path('quadros')
all_image_paths = list(data_root.glob('*/*.jpg'))

In [ ]:
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)

In [ ]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]

In [ ]:
#pre-processamento

def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)  #  carrega as imagens pra memória
  image = tf.image.resize(image, [IMG_W, IMG_H])  #  redimensiona elas
  image = tf.image.rgb_to_grayscale(image)  #  trás pra escala de cinza [W x H x 3] -> [W x H]
  image = tf.reshape(image, (9216,))  #  transforma em um vetor
  image /= 255.0 # normalize to [0,1] range
  return image

def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

In [ ]:
#tensorflow dataset

path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))


In [2]:
#preparando o dataset para ser treinado

BATCH_SIZE = int(image_count * TRAIN_TEST_RATIO)  #para treinar o dataset completo em todas as epocas

test_dataset = image_label_ds.skip(image_count - BATCH_SIZE) 
train_dataset = image_label_ds.take(BATCH_SIZE)

ds = train_dataset.shuffle(buffer_size=BATCH_SIZE)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)

ds = ds.prefetch(buffer_size=AUTOTUNE)

tds = train_dataset.shuffle(buffer_size=image_count-BATCH_SIZE)
tds = tds.repeat()
tds = tds.batch(image_count - BATCH_SIZE)
tds = tds.prefetch(buffer_size=AUTOTUNE)


NameError: name 'image_count' is not defined

In [ ]:
#Definindo o modelo da rede

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(4608, input_dim=9216, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))


In [ ]:
model.compile(
    optimizer = 'rmsprop', 
    #tf.train.AdamOptimizer
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics = ["accuracy"]
)
model.summary()

In [ ]:
#treinando o modelo
H = model.fit(
    ds,  #dataset
    epochs=EPOCHS,  
    steps_per_epoch=int(len(all_image_paths)/BATCH_SIZE),  
)

In [ ]:
score = model.evaluate(tds, verbose=1, steps=int(image_count - BATCH_SIZE))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#Plotando no grafico

plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["acc"], label="train_acc")
# plt.plot(np.arange(0, EPOCHS), H.history["val_acc"], label="val_acc")
# plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()